In [1]:
#include <iostream>
#include <thread>
#include <vector>
#include <iostream>
#include <atomic>
#include <set>
#include <string>
#include "infoapi.hpp"

const int N = 100;
// S(N) = 1 + 2*2 + 3*3 + 4*4 + .. + (N-1)*(N-1) + N*N
{
    int S = 0;
    for(int n = 1; n <= N; n++) S = S + n*n;
    cout << "S(" << N << ") = " << S << endl;
}

S(100) = 338350


In [2]:
class SquareSumObject: public DataObject{
    public:
        SquareSumObject(unsigned pid, EventLog&log):DataObject(pid,log),
        get_number_transaction(*this),put_square_number_transaction(*this){
            my_pid = pid;
            n = 1; S = 0; last_taken = false; last_taken_n = -1;
            numbers_in_sum = 0;
        }
    public:
        bool get_number(int& num){
            get_number_transaction.pid = my_pid;
            get_number_transaction.run();
            if(get_number_transaction.return_val){
                num = get_number_transaction.num;
                return true;
            }
            return false;        
        }
        bool reget_number(int& num){
            update();
            if(last_taken){
                num = last_taken_n;
                return true;
            }
            return false;
        }
        void put_squared_number(int num){
            put_square_number_transaction.pid = my_pid;
            put_square_number_transaction.num = num;
            put_square_number_transaction.run();
        }
        bool get_square_sum(int& sum){
            if(numbers_in_sum==N){
                sum = S; return true;
            }
            return false;
        }
    private: 
        int my_pid;
        int n; int S;
        bool last_taken; int last_taken_n;
        int numbers_in_sum;
    private:
        class GetNumber: public Transaction{
        public:
            GetNumber(SquareSumObject& sq_sum):Transaction(sq_sum),data(sq_sum){}
        public://input
            unsigned pid;
        public://output
            int num;
            bool return_val;
        private:
            void on_run()override{
                if(data.n<=N){
                    num = data.n++; 
                    if(pid==data.my_pid){
                        data.last_taken = true; 
                        data.last_taken_n = num;
                    }
                    return_val = true;
                }
                else
                    return_val = false;
            }
        private:
            SquareSumObject& data;
            void on_save(ostream&out)override{ out << pid;  }
            void on_load(istream&in)override{  in >> pid; }
        } get_number_transaction;
        
        class PutSquaredNumber: public Transaction{
        public:
            PutSquaredNumber(SquareSumObject& sq_sum):Transaction(sq_sum),data(sq_sum){}
        public://input
            int num;
            unsigned pid;
        private:
            void on_run()override{
                data.S = data.S + num;
                data.numbers_in_sum++;
                if(pid==data.my_pid)data.last_taken = false;
            }
        private:
            SquareSumObject& data;
            void on_save(ostream&out)override{ out << num << " " << pid;  }
            void on_load(istream&in)override{  in >> num >> pid; }
        } put_square_number_transaction;
};

In [3]:
// 1 //
{
    atomic_int PID = 0;
    mutex mut;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        int pid = PID++;
    //////////////// inside a process ////////////////
        int num, S;     
        SquareSumObject sq_sum_obj(pid,log);
    
        if(sq_sum_obj.reget_number(num)){
            sq_sum_obj.put_squared_number(num*num);
        }

        while(sq_sum_obj.get_number(num)){
            cout << " number " << num << " taken in process #" << pid << endl;  
            sq_sum_obj.put_squared_number(num*num);
        }

        if(sq_sum_obj.get_square_sum(S)){
            cout << "pid = " << pid << " S(" << N << ") = " << S << endl;
        }
    //////////////////////////////////////////////////
    }); 
    for(auto& t:threads) t.join();
}

 number 1 taken in process #0
 number 2 taken in process #1
 number 12 taken in process #1
 number 13 taken in process #1
 number 14 taken in process #1
 number 15 taken in process #1
 number 16 taken in process #1
 number 17 taken in process #1
 number 18 taken in process #1
 number 19 taken in process #1
 number 20 taken in process #1
 number 21 taken in process #1
 number 22 taken in process #1
 number 23 taken in process #1
 number 24 taken in process #1
 number 25 taken in process #1
 number 26 taken in process #1
 number 27 taken in process #1
 number 28 taken in process #1
 number 29 taken in process #1
 number 30 taken in process #1
 number 31 taken in process #1
 number 32 taken in process #1
 number 33 taken in process #1
 number 34 taken in process #1
 number 35 taken in process #1
 number 36 taken in process #1
 number 37 taken in process #1
 number 38 taken in process #1
 number 39 taken in process #1
 number 40 taken in process #1
 number 41 taken in process #1
 number 42